# Домашнее задание №3

<span style="color: red; font-size: 14pt">Дедлайн: 20 марта 23:59</span>

Машинное обучение, ФИВТ, Весна 2018

**Оформление дз**: 
- Присылайте выполненное задание на почту ``ml.course.mipt@gmail.com``
- Укажите тему письма в следующем формате ``ML2018_fall <номер_группы> <фамилия>``, к примеру -- ``ML2018_fall 596 ivanov``
- Выполненное дз сохраните в файл ``ML2018_<фамилия>_<группа>_task<номер задания>.ipnb``, к примеру -- ``ML2018_ivanov_596_task1.ipnb``

**Вопросы**:
- Присылайте вопросы на почту ``ml.course.mipt@gmail.com`` (или в телеграм-канал)
- Укажите тему письма в следующем формате ``ML2018_fall Question <Содержание вопроса>``

--------
- **PS1**: Используются автоматические фильтры, мы не найдем ваше дз, если вы укажете тему письма в неправильном формате.
- **PS2**: Просроченный дедлайн снижает максимальный вес задания по формуле, указнной на первом семинаре

## Часть 1. Теоретические задачи

##### 30% баллов за задание, оценочное время выполнения 30 минут

### Задача 1 (10% баллов)

Предположим, что мы решаем задачу бинарной классификации и что у нас есть три алгоритма $b_1(x)$, $b_2(x)$ и $b_3(x)$, каждый из которых ошибается с вероятностью p. Мы строим композицию взвешенным голосованием: алгоритмам присвоены значимости $w_1$, $w_2$ и $w_3$, и для вынесения вердикта суммируются значимости алгоритмов, проголосовавших за каждый из классов:

$$a_0 = \sum_{i=1}^3 w_i [b_i(x)=0]$$
$$a_1 = \sum_{i=1}^3 w_i [b_i(x)=1]$$


Объект $x$ относится к классу, для которого такая сумма оказалась максимальной. Например, если первые два алгоритма голосуют за класс $0$, а третий — за класс $1$, то выбирается класс $0$, если $w_1 + w_2 > w_3$, и класс $1$ в противном случае. Какова вероятность ошибки такой композиции этих трех алгоритмов, если:
1. $w_1 = 0.2, w_2 = 0.3, w_3 = 0.2$;
2. $w_1 = 0.2, w_2 = 0.5, w_3 = 0.2$? 

1) $$w_1 + w_3 > w_2$$
$$w_1 + w_2 > w_3$$
$$w_2 + w_3 > w_1$$
Следовательно, $$3\cdot( p \cdot p \cdot (1 - p) ) + p^3 = 3\cdot p^2 \cdot (1 - p) + p^3$$

2) $$ w_1 + w_2 > w_3$$

$$w_2 + w_3 > w_1$$

$$w_2 > w_1 + w_3 $$
Следовательно,
$$2\cdot p^2 \cdot (1 - p) + p^3 + p\cdot(1 - p)^2 = p$$

### Задача 2 (10% баллов)

Рассмотрим задачу бинарной классификации. Будем считать, что все алгоритмы из базового семейства возвращают ответы из отрезка $[0,1]$, которые можно интерпретировать как вероятности принадлежности объектов классу $1$. В качестве функции потерь возьмем отрицательный логарифм правдоподобия:
$$L(y,z) = -(y \log{z}+(1-y)\log{(1-z)})$$
В формуле $y$ - правильный ответ, $z$ - ответ алгоритма. Выпишите формулы для поиска базовых алгоритмов $b_n$ и коэффициентов $\gamma_n$ в градиентном бустинге.

In [2]:
# Ваш ответ здесь

### Задача 3 (10% баллов)

Известно, что на $n$-й итерации двухклассового метода AdaBoost
был выбран базовый классификатор, допускающий ошибку только на одном объекте $x_j$. Найдите нормированный вес $w_j^{(n+1)}$ при этом объекте на следующей итерации.

Так как ошибается только на $x_j$:
$$\alpha^{(n)} = \frac{1}{2} ln(\frac{1 - w_j}{w_j})$$
Подставим:
$$w_{j}^{(n + 1)} = \frac{w_{j}^{(n)}\exp{(\alpha^{(n)})}}{\exp{(-\alpha^{(n)})} \cdot \sum_{i \neq j}w_{i}^{(n)} + \exp{(\alpha^{(n)})}\cdot w_{j}^{(n)}}$$
    
$$ \sum_{i \neq j}w_{i}^{(n)} = 1 - w_{j}^{(n)}$$

$$\exp{(\alpha^{(n)})} = \sqrt{\frac{1 - w_{j}^{(n)}}{w_{j}^{(n)}}}$$

Поделим числитель и знаменатель дроби на числитель:
$$w_{j}^{(n + 1)} = \frac{1}{\frac{1 - w_{j}^{(n)}}{1 - w_{j}^{(n)}} + 1} = \frac{1}{2}$$

## Часть 2. Random Forest

##### 70% баллов за задание, оценочное время выполнения 3 часа

### Реализация (40%)

**Необходимо реализовать класс `RandomForest`** (для решения задачи классификации)

**Спецификация:**
- класс наследуется от `sklearn.BaseEstimator`;
- конструктор содержит следующие параметры: 
    - `num_trees` - количество деревьев в лесе;
    - `max_depth` - максимальная глубина дерева (по умолчанию - `numpy.inf`); 
    - `max_features` - количество признаков, принимаемое к рассмотрению при разбиении (аналогичный параметр есть в sklearn имплементации). Параметр может принимать значения:
        - int - тогда рассматриваем max_features признаков при каждом разбиении;
        - float - max_features обозначает процент, int(max_features * n_features) признаков рассматривается при каждом разбиении;
        - “sqrt” - max_features=sqrt(n_features);
        - “log2” - max_features=log2(n_features);
        - None - max_features=n_features;
    - `criterion` - критерий разбиения (для классификации - 'gini' или 'entropy', по умолчанию - 'gini'); функции с подсчетом энтропийного и критерия Джини можно взять из предыдущего дз;
    
- класс имеет методы `fit` и `predict`;
- метод `fit` принимает матрицу объектов `X` и вектор ответов `y` (объекты `numpy.ndarray`) и возвращает экземпляр класса
    `RandomForest`, представляющий собой Random Forest, обученный по выборке `(X, y)` с учётом заданных в конструкторе параметров; 
- метод `predict` принимает матрицу объектов и возвращает вектор предсказанных ответов;

In [4]:
import random
import sklearn
from sklearn import tree
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score

In [5]:
random.seed(None)

In [6]:
def RSM(X, y, size):
    features = []
    for i in range(size):
        features.append(random.randint(0, X.shape[1] - 1))
    features = list(set(features))
    sample_X, sample_y = X[:, features], y
    return sample_X, sample_y, features

In [7]:
def bagging(X, y, size):
    ind = []
    for i in range(size):
        ind.append(random.randint(0, X.shape[0] - 1))
    sample_X, sample_y = X[ind, :], y[ind]
    return sample_X, sample_y

In [8]:
class RandomForestMod(sklearn.base.BaseEstimator, sklearn.base.ClassifierMixin):
    def __init__(self, num_trees, max_depth=None, max_features=None, criterion='gini'):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.trees = []
        self.features = []
        self.max_features = max_features
        self.criterion = criterion
        
    def _set_max_features_(self):        
        if isinstance(self.max_features, float):
            self.max_features = self.max_features*self.n_features
        elif isinstance(self.max_features, str):
            if self.max_features == 'sqrt':
                self.max_features = np.sqrt(self.n_features)
            elif self.max_features == 'log2':
                self.max_features = np.log2(self.n_features)
        elif self.max_depth is None:
            self.max_features = self.n_features
    
    def fit(self, X_train, y_train):
        self.n_features = X_train.shape[1]
        self._set_max_features_()
        for i in range(self.num_trees):
            X, y, features = RSM(X_train, y_train, self.max_features)
            model = tree.DecisionTreeClassifier(criterion=self.criterion, max_depth = self.max_depth)
            model.fit(X, y)
            self.trees.append(model)
            self.features.append(features)
        return self
    
    def predict(self, X_test):
        res = []
        for tree, i in zip(self.trees, range(self.num_trees)):
            res.append(tree.predict(X_test[:, self.features[i]]))
            
        res = np.array(res)
        y_pred = np.zeros(X_test.shape[0], dtype=np.int)
        
        for col in range(X_test.shape[0]):
            y_pred[col] = np.bincount(res[:, col]).argmax()

        return y_pred

In [9]:
class RandomForest(sklearn.base.BaseEstimator, sklearn.base.ClassifierMixin):
    def __init__(self, num_trees, max_depth=None, max_features=None, criterion='gini'):
        self.num_trees = num_trees
        self.max_depth = max_depth
        self.trees = []
        self.max_features = max_features
        self.criterion = criterion
        
    def _set_max_features_(self):        
        if isinstance(self.max_features, float):
            self.max_features = self.max_features*self.n_features
        elif isinstance(self.max_features, str):
            if self.max_features == 'sqrt':
                self.max_features = np.sqrt(self.n_features)
            elif self.max_features == 'log2':
                self.max_features = np.log2(self.n_features)
        elif self.max_depth is None:
            self.max_features = self.n_features
    
    def fit(self, X_train, y_train):
        self.n_features = X_train.shape[1]
        self._set_max_features_()
        for i in range(self.num_trees):
            X, y = bagging(X_train, y_train, X_train.shape[0])
            model = tree.DecisionTreeClassifier(criterion=self.criterion, max_depth = self.max_depth, max_features=self.max_features)
            model.fit(X, y)
            self.trees.append(model)
        return self
    
    def predict(self, X_test):
        res = []
        for tree, i in zip(self.trees, range(self.num_trees)):
            res.append(tree.predict(X_test))
            
        res = np.array(res)
        y_pred = np.zeros(X_test.shape[0], dtype=np.int)
        
        for col in range(X_test.shape[0]):
            y_pred[col] = np.bincount(res[:, col]).argmax()

        return y_pred

### Тестирование (15%)

Загрузите датасет Wine Data Set (https://archive.ics.uci.edu/ml/datasets/wine). Разделите выборку на обучающую и тестовую с помощью метода `train_test_split`, используйте значения параметров `test_size=0.2`, `random_state=42`. Попробуйте обучить Random Forest на предложенном датасете

In [10]:
data = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/wine/wine.data', header=None)
columns = ['class', 'Alcohol', 'Malic acid', 'Ash', 'Alcalinity of ash', 'Magnesium', 'Total phenols', 'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins', 'Color intensity', 'Hue', 'OD280/OD315 of diluted wines', 'Proline']
data.columns = columns
data.head()

,class,Alcohol,Malic acid,Ash,Alcalinity of ash,Magnesium,Total phenols,Flavanoids,Nonflavanoid phenols,Proanthocyanins,Color intensity,Hue,OD280/OD315 of diluted wines,Proline
0,1,14.23,1.71,2.43,15.6,127,2.80,3.06,0.28,2.29,5.64,1.04,3.92,1065
1,1,13.20,1.78,2.14,11.2,100,2.65,2.76,0.26,1.28,4.38,1.05,3.40,1050
2,1,13.16,2.36,2.67,18.6,101,2.80,3.24,0.30,2.81,5.68,1.03,3.17,1185
3,1,14.37,1.95,2.50,16.8,113,3.85,3.49,0.24,2.18,7.80,0.86,3.45,1480
4,1,13.24,2.59,2.87,21.0,118,2.80,2.69,0.39,1.82,4.32,1.04,2.93,735


In [11]:
train, test = train_test_split(data, test_size=0.2, random_state=42)
X_train, y_train = np.array(train[columns[1:]]), np.array(train['class'])
X_test, y_test = np.array(test[columns[1:]]), np.array(test['class'])

In [12]:
model = RandomForest(num_trees=20, max_features=5)
model.fit(X_train, y_train)

RandomForest(criterion='gini', max_depth=None, max_features=13, num_trees=20)

In [13]:
acc = []
for k in range(100):
    y_pred = model.predict(X_test)
    acc.append(accuracy_score(y_pred, y_test))
print(np.mean(acc))

1.0


Покажите, как менялись значения критерия качества `accuracy` при увеличении параметра num_trees. Видны ли следы переобучения?

In [14]:
acc = []
for n in range(10, 1000, 100):
    for k in range(100):
        model = RandomForest(num_trees=n, max_features=5)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc.append(accuracy_score(y_pred, y_test))
    print("n: {}".format(n), np.mean(acc))

n: 10 0.9686111111111111
n: 110 0.9699999999999998
n: 210 0.9705555555555556
n: 310 0.970972222222222
n: 410 0.9712222222222221
n: 510 0.9713888888888887
n: 610 0.9715079365079363
n: 710 0.971597222222222
n: 810 0.9716666666666665
n: 910 0.9717222222222219


Сравните качество работы вашей реализации RandomForest и реализации из sklearn.

In [15]:
from sklearn.ensemble import RandomForestClassifier

rfc = RandomForestClassifier(random_state=42, n_jobs=-1)
rfc.fit(X_train, y_train)   
print(accuracy_score(rfc.predict(X_test), y_test))

0.9444444444444444


### Модификация Random Forest (15%)

Измените свою реализацию `RandomForest` так, чтобы случайное подмножество признаков выбиралось не в каждом сплите, а перед построением всего дерева. Сравните результат работы с обычным RandomForest.

In [16]:
model = RandomForestMod(num_trees=20, max_features=5)
model.fit(X_train, y_train)

acc = []
for k in range(100):
    y_pred = model.predict(X_test)
    acc.append(accuracy_score(y_pred, y_test))
print(np.mean(acc))

1.0


In [17]:
acc = []
for n in range(10, 1000, 100):
    for k in range(100):
        model = RandomForestMod(num_trees=n, max_features=5)
        model.fit(X_train, y_train)
        y_pred = model.predict(X_test)
        acc.append(accuracy_score(y_pred, y_test))
    print("n: {}".format(n), np.mean(acc))

n: 10 0.9694444444444444
n: 110 0.9727777777777775
n: 210 0.9736111111111112
n: 310 0.9735416666666664
n: 410 0.9734999999999998
n: 510 0.973287037037037
n: 610 0.9731746031746031
n: 710 0.9730555555555555
n: 810 0.9729629629629628
n: 910 0.9728888888888886
